In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import sys
sys.path.append('../')

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd

DATA_PATH = '../data/'
GENERATED_PATH = '../generated/'
GLD_LCIM_STATISTICS_PATH = f'{DATA_PATH}gld_lcim/statistics.npy'

In [58]:
# Load statistics
statistics = np.load(GLD_LCIM_STATISTICS_PATH, allow_pickle=True).item()

# Apply median for each statistic
for scene_key in statistics.keys():
    for statistic_key in statistics[scene_key].keys():
        statistics[scene_key][statistic_key] = np.median(statistics[scene_key][statistic_key])

# Get diff statistics
part2_diff_statistics = {}
for scene_key in statistics.keys():
    if 'P2' not in scene_key:
        continue
    scene_type = scene_key[2:-6].lower()
    scene_id = f'p2_{int(scene_key[-6:-4])}'
    for statistic_key in statistics[scene_key].keys():
        if scene_id not in part2_diff_statistics:
            part2_diff_statistics[scene_id] = {}
        value = statistics[scene_key][statistic_key]
        value *= 1 if scene_type == 'clear' else -1
        if statistic_key not in part2_diff_statistics[scene_id]:
            part2_diff_statistics[scene_id][statistic_key] = 0
        part2_diff_statistics[scene_id][statistic_key] += value

# Reformat scene names from PXSceneX to pX_X
statistics = {
    f"p{k[1:2]}_{int(k[-6:-4])}{'_o' if 'Overcast' in k else '_c' if 'Clear' in k else ''}": v for k, v in statistics.items()}


In [61]:
stats = pd.DataFrame(statistics).T
stats.head(50)
stats.to_csv(f'{GENERATED_PATH}gld_lcim/statistics.csv')

In [62]:
# As pandas
part_diff_stats = pd.DataFrame(part2_diff_statistics).T
part_diff_stats.head()
part_diff_stats.to_csv(f'{GENERATED_PATH}gld_lcim/part2_diff_statistics.csv')